# 전처리

In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv("../dataset/240128_TripAdvisorHotelReviews/dataset_2B_removed_stopwords_ver.csv")

### 토큰들을 내부 단어 사전에 매핑하여 딕셔너리 생성

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 토큰화된 텍스트 리스트
# 토크나이저 객체 생성
tokenizer = Tokenizer()

# 토큰들을 내부 단어 사전에 매핑하여 딕셔너리 생성
tokenizer.fit_on_texts(df['tokens'])

# 딕셔너리 확인
word_index = tokenizer.word_index
#print("단어 사전:")
#print(word_index)


2024-02-08 16:51:09.290420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 학습시킨 tokenizer를 바탕으로 토큰들을 시퀀스로 변경

In [5]:
# 토큰들을 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(df['tokens'])

# 변환된 시퀀스 확인
print("\n변환된 시퀀스:")
print(sequences[0])


변환된 시퀀스:
[363, 59725, 68, 303, 23687, 703, 100, 69, 7020, 1542, 6216, 24, 19, 68, 136, 3, 175, 506, 9940, 1561, 3, 40, 985, 120, 997, 31, 719]


In [6]:
len(sequences[1])

31

### 패딩 처리

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# 예시 데이터 (refined_df['tokens']에 토큰이 이미 들어가 있다고 가정)
# 토큰화된 데이터가 저장된 컬럼 이름이 'tokens'일 경우:
tokenized_text = df['tokens']

# 토크나이저 객체 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)

# 토큰들을 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(tokenized_text)

# 시퀀스 패딩
max_sequence_length = 200  # 시퀀스의 최대 길이 (임의 설정)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

### 임베딩 파일들을 word_to_embedding 딕셔너리로 제작

In [8]:
import torch
embedding_file = "Embedding_model/glove.6B.50d.txt"

# 단어와 임베딩을 저장할 딕셔너리 생성
word_to_embedding = {}

# 임베딩 파일을 읽어서 딕셔너리에 저장
with open(embedding_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            values = line.split()
            word = values[0]
            embedding = torch.tensor([float(val) for val in values[1:]])
            word_to_embedding[word] = embedding
        except ValueError:
            continue  # ValueError가 발생하면 해당 행을 건너뜁니다.

In [9]:
sequences = df['tokens']

### Glove로 임베딩

In [10]:
# 임베딩된 시퀀스 저장할 리스트 생성

from tqdm import tqdm
embedded_sequences = []

# 각 토큰에 대해 GloVe 임베딩을 추출하여 리스트에 저장
for sequence in tqdm(padded_sequences, desc="Processing sequences", unit="sequence"):
    embedded_sequence = [word_to_embedding.get(token, torch.zeros(300)) for token in sequence]
    embedded_sequences.append(embedded_sequence)


Processing sequences:   0%|          | 0/200000 [00:00<?, ?sequence/s]

Processing sequences: 100%|██████████| 200000/200000 [07:36<00:00, 438.37sequence/s]  


In [ ]:
labels = df['ratings']

### df['ratings']에 있는 'low', 'high'를 0, 1로 바꾸기

In [21]:
import numpy as np

# labels 배열
labels = np.array(labels)

# 값을 숫자로 변환하는 코드
labels = np.array([1 if value == 'high' else 0 for value in labels])

print("Modified labels:", labels)


Modified labels: [1 1 1 ... 0 0 0]


### 훈련-테스트 데이터셋으로 분리하기 전 임베딩 처리한 값을 Np.array형식으로 바꿔주기

In [23]:
embedded_sequences = np.array(embedded_sequences)

: 

# Train-Test 데이터 분리

In [174]:
from sklearn.model_selection import train_test_split

# PyTorch DataLoader로 사용할 데이터셋 클래스를 정의하고 데이터를 생성하는 작업이 필요

# 예시: X는 embedded_sequence, y는 refined_df['ratings']
X_train, X_temp, y_train, y_temp = train_test_split(embedded_sequences, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 결과 확인
print("Train data shapes:", X_train.shape, y_train.shape)
print("Validation data shapes:", X_val.shape, y_val.shape)
print("Test data shapes:", X_test.shape, y_test.shape)

Train data shapes: (7000, 200, 300) (7000,)
Validation data shapes: (1500, 200, 300) (1500,)
Test data shapes: (1500, 200, 300) (1500,)


# 모델 학습

In [175]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 데이터셋을 플랫하게 펼침
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_val.reshape(X_val.shape[0], -1)


In [64]:
X_train.shape

(7000, 200, 300)

In [176]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# 모델 초기화
model = LogisticRegression()

# 모델 학습
model.fit(X_train_flat, y_train)


LogisticRegression()

In [177]:
# 예측
y_pred = model.predict(X_val_flat)

# 정확도 평가
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.4866666666666667


In [65]:
from sklearn.ensemble import RandomForestClassifier

# 최적의 하이퍼파라미터
best_hyperparameters = {'max_depth': None, 'n_estimators': 150}

# RandomForestClassifier 초기화
model = RandomForestClassifier(**best_hyperparameters)



model.fit(X_train_flat, y_train)

RandomForestClassifier(n_estimators=150)

In [69]:
X_test_flat = X_val.reshape(X_test.shape[0], -1)

# 예측
y_pred = model.predict(X_test_flat)



# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.4886666666666667
